In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
values = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']
cutoff_list = ~application_df["CLASSIFICATION"].isin(values)
classifications_to_replace = application_df["CLASSIFICATION"][cutoff_list]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_data = pd.get_dummies(application_df, dtype=int)
categorical_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
# Split our preprocessed data into our features and target arrays
X = categorical_data.drop("IS_SUCCESSFUL", axis=1)
y = categorical_data["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization Attempt #1 - Tuner

In [40]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=100,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics="accuracy")

    return nn_model

In [42]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [43]:
# run kerastuner to search for best parameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 49s]
val_accuracy: 0.731195330619812

Best val_accuracy So Far: 0.7336443066596985
Total elapsed time: 00h 20m 37s


In [44]:
# get best model hyperparameters
best_param = tuner.get_best_hyperparameters(1)[0]
best_param.values

{'activation': 'sigmoid',
 'first_units': 20,
 'num_layers': 4,
 'units_0': 71,
 'units_1': 13,
 'units_2': 61,
 'units_3': 87,
 'units_4': 3,
 'units_5': 73,
 'units_6': 37,
 'units_7': 53,
 'units_8': 9,
 'units_9': 25,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [45]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5723 - accuracy: 0.7336 - 389ms/epoch - 1ms/step
Loss: 0.5723136067390442, Accuracy: 0.7336443066596985


Optimization Method 2: Random Forest Feature Importances

In [46]:
# create a random forest classifier instance
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=500, random_state=1)

In [49]:
# train the random forest model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [50]:
# calculate feature importances to optimize model
importances = rf_model.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.4044862027378769, 'ASK_AMT'),
 (0.1295399971904381, 'AFFILIATION_CompanySponsored'),
 (0.11975571666568592, 'AFFILIATION_Independent'),
 (0.03003574529731091, 'APPLICATION_TYPE_T5'),
 (0.027241088586899446, 'ORGANIZATION_Association'),
 (0.023455419488934134, 'APPLICATION_TYPE_T10'),
 (0.020491933685826684, 'ORGANIZATION_Trust'),
 (0.01903833985767598, 'CLASSIFICATION_Other'),
 (0.018745766723289467, 'CLASSIFICATION_C2100'),
 (0.018528250593078416, 'APPLICATION_TYPE_T19'),
 (0.017550939744189753, 'APPLICATION_TYPE_T4'),
 (0.01690465537583236, 'CLASSIFICATION_C2000'),
 (0.01685295991976745, 'APPLICATION_TYPE_T3'),
 (0.014421840527727036, 'CLASSIFICATION_C1000'),
 (0.012624778759431137, 'USE_CASE_ProductDev'),
 (0.011998308436650956, 'APPLICATION_TYPE_T6'),
 (0.011105003585320355, 'USE_CASE_Preservation'),
 (0.009298767898046107, 'INCOME_AMT_0'),
 (0.008823683788573396, 'CLASSIFICATION_C1200'),
 (0.008568098976315403, 'CLASSIFICATION_C3000'),
 (0.007185690044691464, 'INCOME_AMT_25000

It looks like Status and Special Considerations are the features with the lowest importance. I am going to drop those from the original dataframe.

In [51]:
# Create a copy of the original dataframe and drop columns to try to optimize the model

test_df = application_df.copy()
test_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"], inplace=True)

In [52]:
# view dataframe
test_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [59]:
# make sure data is still binned in new df
test_df["CLASSIFICATION"].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [55]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_data_test = pd.get_dummies(test_df, dtype=int)

In [56]:
# split preprocessed data into target and features
X_2 = categorical_data_test.drop("IS_SUCCESSFUL", axis=1)
y_2 = categorical_data_test["IS_SUCCESSFUL"]

In [57]:
# split preprocessed data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, random_state=1)

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [65]:
# define the model and create based on original attempt
input_features = len(X_train_scaled[0])
nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim = input_features))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=30, activation="relu"))

# output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check structure
nn_model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3280      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5741 (22.43 KB)
Trainable params: 5741 (22.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
# compile the model 
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [67]:
# train the new model
train_model_2 = nn_model_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5706 - accuracy: 0.7195
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7316
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7336
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accura

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7305 - 358ms/epoch - 1ms/step
Loss: 0.5576905608177185, Accuracy: 0.7304956316947937


Optimization Method 3 - Combine feature importances with best params from tuner

In [75]:
# Redefine method that creates a new Sequential model with hyperparameter options (fewer input features)
def create_model_opt(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=100,
        step=5), activation=activation, input_dim=40))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics="accuracy")

    return nn_model

In [76]:
# reinitiate tuner using defined function
tuner = kt.Hyperband(
    create_model_opt,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [77]:
# run kerastuner to search for best parameters - will use data from optimization attempt 2 with less important features removed
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 26 Complete [00h 00m 34s]
val_accuracy: 0.7323614954948425

Best val_accuracy So Far: 0.7335277199745178
Total elapsed time: 00h 05m 36s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
50                |50                |first_units
9                 |6                 |num_layers
1                 |45                |units_0
3                 |39                |units_1
7                 |27                |units_2
29                |61                |units_3
57                |69                |units_4
57                |67                |units_5
1                 |81                |units_6
5                 |35                |units_7
69                |93                |units_8
75                |27                |units_9
20                |7                 |tuner/epochs
0                 |3                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0      

In [ ]:
# get best model hyperparameters
best_param = tuner.get_best_hyperparameters(1)[0]
best_param.values

268/268 - 0s - loss: 0.5534 - accuracy: 0.7268 - 371ms/epoch - 1ms/step
Loss: 0.5534236431121826, Accuracy: 0.7267638444900513


In [ ]:
# Evaluate best model against full test data
opt_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = opt_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# save optimized model
opt_model.save('AlphabetSoupCharity_Optimization.h5')